In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
import os

def safe_load_img(file_path):
    try:
        img = load_img(file_path)
        return img
    except (IOError, OSError):
        print(f"Skipping corrupted file: {file_path}")
        return None

def preprocess_image(file_path):
    img = safe_load_img(file_path)
    if img:
        img = img.convert("RGB")
        return img
    return None

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_image,
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_image,
    rescale=1./255
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 946 images belonging to 5 classes.
Found 453 images belonging to 5 classes.


In [4]:
from PIL import Image
import os

def verify_image(file_path):
    try:
        img = Image.open(file_path)
        img.verify()  # Verify that it is, in fact, an image
        img.close()  # Close the image file
        img = Image.open(file_path)
        img.load()  # Ensure it loads properly
        img.close()
        return True
    except (IOError, OSError):
        return False

def check_and_remove_corrupted_images(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            if not verify_image(file_path):
                print(f"Removing corrupted file: {file_path}")
                os.remove(file_path)

# Replace 'your_dataset_directory' with your actual dataset directory
check_and_remove_corrupted_images('data')


In [2]:
# Set paths
train_dir = 'data/train'
test_dir = 'data/test'

In [5]:
# Data Augmentation and Preprocessing
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')

Found 946 images belonging to 5 classes.
Found 453 images belonging to 5 classes.


In [7]:
# Load Pre-trained MobileNetV2 Model + Higher Layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(5, activation='softmax')(x)  # Assuming 5 yoga poses

In [8]:
model = Model(inputs=base_model.input, outputs=predictions)

In [9]:
# Compile the Model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
print(train_generator.class_indices)
print(test_generator.class_indices)

{'downdog': 0, 'goddess': 1, 'plank': 2, 'tree': 3, 'warrior': 4}
{'downdog': 0, 'goddess': 1, 'plank': 2, 'tree': 3, 'warrior': 4}


In [11]:
# Train the Model
model.fit(train_generator, epochs=10, validation_data=test_generator)

Epoch 1/10


C:\Users\mrsta\AppData\Roaming\Python\Python310\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
C:\Program Files\Python310\lib\site-packages\PIL\Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


30/30 ━━━━━━━━━━━━━━━━━━━━ 103s 3s/step - accuracy: 0.4403 - loss: 1.3725 - val_accuracy: 0.7550 - val_loss: 0.6669
Epoch 2/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 84s 3s/step - accuracy: 0.8841 - loss: 0.4130 - val_accuracy: 0.9183 - val_loss: 0.2374
Epoch 3/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 85s 3s/step - accuracy: 0.9493 - loss: 0.1871 - val_accuracy: 0.8675 - val_loss: 0.3194
Epoch 4/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 85s 3s/step - accuracy: 0.9518 - loss: 0.1510 - val_accuracy: 0.8631 - val_loss: 0.3852
Epoch 5/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 84s 3s/step - accuracy: 0.9765 - loss: 0.0945 - val_accuracy: 0.8212 - val_loss: 0.5832
Epoch 6/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 85s 3s/step - accuracy: 0.9890 - loss: 0.0416 - val_accuracy: 0.8256 - val_loss: 0.4961
Epoch 7/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 85s 3s/step - accuracy: 0.9906 - loss: 0.0360 - val_accuracy: 0.8962 - val_loss: 0.3490
Epoch 8/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 86s 3s/step - accuracy: 0.9899 - loss: 0.0332 - val_accuracy: 0.8918 - val_loss: 0.3908
Ep

In [13]:
# Save the Model
model.save('model/yoga_pose_model_final.h5')